In [ ]:
import json
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer 

questions_file_path = "/kaggle/input/benchmark-techqa/benchmark.json"
rewrited_questions_file_path = "/kaggle/working/benchmark_query_rewriting.json"

# Open input file
with open(questions_file_path, "r") as benchmark_file:
    benchmark_instances = json.load(benchmark_file)

# Set GPU if available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# Load query rewriting model
query_rewriter_model_name = "catyung/t5l-turbo-hotpot-0331"
model = T5ForConditionalGeneration.from_pretrained(query_rewriter_model_name)
tokenizer = T5Tokenizer.from_pretrained(query_rewriter_model_name)

quest_cont = 0
new_benchmark_instances = []

for benchmark_instance in benchmark_instances:
    if quest_cont % 50 == 0:
        print(str(quest_cont))
    quest_cont = quest_cont + 1
    
    # Give a prompt to the model for rewriting the question
    user_query = benchmark_instance["question"]
    rewrite_prompt = f"""rewrite a better search query: {user_query}
    answer:"""
    input_ids = tokenizer(rewrite_prompt, return_tensors="pt").input_ids.to(device)
    outputs = model.generate(input_ids, max_new_tokens=100)
    rewrited_question = tokenizer.decode(outputs[0], skip_special_tokens=True)

    new_benchmark_instance = benchmark_instance
    new_benchmark_instance["rewrited_question"] = rewrited_question
    new_benchmark_instances.append(new_benchmark_instance)
    
with open(rewrited_questions_file_path, "w") as file:
    json.dump(new_benchmark_instances, file, indent=4)